In [1]:
import io
import xml.etree.ElementTree as ET
import pandas as pd
import helper
from sklearn.feature_extraction.text import CountVectorizer
import gensim
import numpy as np
import os
from gensim import corpora, models, similarities 

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%install_ext https://raw.github.com/cpcloud/ipython-autotime/master/autotime.py
%load_ext autotime

Installed autotime.py. To use it, type:
  %load_ext autotime


In [4]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

time: 3 ms


In [5]:
ls data

 Volume in drive E has no label.
 Volume Serial Number is 19C7-A2B3

 Directory of E:\metis\ds4_fletcher\data

08/17/2015  12:55 PM    <DIR>          .
08/17/2015  12:55 PM    <DIR>          ..
03/09/2015  06:04 PM        60,201,826 Badges.xml
03/09/2015  06:04 PM       238,716,194 Comments.xml
03/09/2015  06:05 PM     1,144,804,160 PostHistory.xml
03/09/2015  06:06 PM         6,192,845 PostLinks.xml
03/09/2015  06:06 PM       736,314,717 Posts.xml
03/09/2015  06:06 PM           367,147 Tags.xml
03/09/2015  06:06 PM       100,081,558 Users.xml
03/09/2015  06:07 PM       210,233,404 Votes.xml
               8 File(s)  2,496,911,851 bytes
               2 Dir(s)  638,569,201,664 bytes free
time: 18 ms


In [6]:
data_dir='data'

time: 2 ms


## Normalizer

In [7]:
ag_preprocesser = helper.Normalizer(stem=False\
                                 ,stopword_removal=True\
                                 ,strip_html_tags=True)

time: 3 ms


## Loading Streaming

In [8]:
post_file=os.path.join(data_dir,'Posts.xml')
posts_corpus = helper.XmlCorpus(post_file
                                ,clip_docs=10
                                ,body_field='Body'
                                ,normalizer=ag_preprocesser)

time: 2 ms


In [9]:
sample_msg='<p>A Vista virtual machine I use only has a 10 GB virtual hard disk -- and I\'m worried about it running out of space over time.</p>\n\n<p>I used <a href="http://www.jam-software.com/treesize_free/">TreeSize</a> to check for the culprits..</p>\n\n<p><img src="http://i.stack.imgur.com/uGfhQ.png" alt="enter image description here"></p>\n\n<p>And the primary culprit is that <strong><code>/winsxs</code> or "Windows Side-By-Side" folder</strong>.</p>\n\n<p><img src="http://i.stack.imgur.com/DWYOo.png" alt="enter image description here"></p>\n\n<p>What is the Windows Side By Side folder? Well, that\'s <a href="http://blogs.technet.com/askcore/archive/2008/09/17/what-is-the-winsxs-directory-in-windows-2008-and-windows-vista-and-why-is-it-so-large.aspx">explained pretty well here</a>.</p>\n\n<blockquote>\n  <p>All of the components in the operating system are found in the WinSxS folder \u2013 in fact we call this location the component store.  Each component has a unique name that includes the version, language, and processor architecture that it was built for.  The WinSxS folder is the only location that the component is found on the system, all other instances of the files that you see on the system are \u201cprojected\u201d by hard linking from the component store.  Let me repeat that last point \u2013 there is only one instance (or full data copy) of each version of each file in the OS, and that instance is located in the WinSxS folder.   So looked at from that perspective, the WinSxS folder is really the entirety of the whole OS, referred to as a "flat" in down-level operating systems.  This also accounts for why you will no longer be prompted for media when running operations such as System File Checker (SFC), or when installing additional features and roles.</p>\n  \n  <p>That explains why the folder starts off big, but not why it gets larger over time \u2013 the answer to that question is servicing.   In previous versions of Windows the atomic unit of servicing was the file, in Windows Vista it\u2019s the component.  When we update a particular binary we release a new version of the whole component, and that new version is stored alongside the original one in the component store.  The higher version of the component is projected onto the system, but the older version in the store isn\u2019t touched.  The reason for that is the third part of why the component store gets so large.</p>\n</blockquote>\n\n<p>Lots of other people <a href="http://blog.tiensivu.com/aaron/archives/1306-Demystifying-the-WinSxS-directory-in-Windows-XP,-Vista-and-Server-20032008.html">complaining about the size of the /winsxs folder, too</a>:</p>\n\n<blockquote>\n  <p>A coworker recently received his every two year work laptop refresh and was in the middle of loading his new laptop with Vista. For some reason, his hard drive C: was partitioned to only 25GB and the rest of the space reserved for a much larger drive D:.</p>\n  \n  <p>Anyway, after loading many Windows Updates, he was left with hardly any drive space left on C:, and for some reason, the WinSxS directory had ballooned to over 8GB.</p>\n  \n  <p>While I knew what the WinSxS (Windows Side-by-Side) directory was for, I wasn\'t quite sure why that directory would explode in size so fast with normal operating system updates.</p>\n</blockquote>\n\n<p>I don\'t really care about having all these duplicate files for safety, but I <em>do</em> care about the VM exploding due to an out of disk space error. So.. <strong>can /winsxs be safely pared down in any way?</strong> <a href="http://blogs.msdn.com/jonwis/archive/2007/01/02/deleting-from-the-winsxs-directory.aspx">It doesn\'t seem that it can be</a>.. </p>\n\n<blockquote>\n  <p><strong>Administrators should not, for any reason, take it upon themselves to clean out the /winsxs directory</strong> - doing so may prevent Windows Update and MSI from functioning properly afterwards.  Preventing accidental deletion from the directory is accomplished by putting a strong security descriptor on the directory that inherits to its children.</p>\n</blockquote>\n\n<p>If this /winsxs folder gets any larger, I\'m going to have to re-image this virtual machine.. which I <em>really</em> don\'t want to do!</p>\n'

ag_preprocesser.process(sample_msg)

[u'A',
 u'Vista',
 u'virtual',
 u'machine',
 u'I',
 u'use',
 u'GB',
 u'virtual',
 u'hard',
 u'disk',
 u'I',
 u"'m",
 u'worried',
 u'running',
 u'space',
 u'time',
 u'I',
 u'used',
 u'TreeSize',
 u'check',
 u'culprits..',
 u'And',
 u'primary',
 u'culprit',
 u'/winsxs',
 u'Windows',
 u'Side-By-Side',
 u'folder',
 u'What',
 u'Windows',
 u'Side',
 u'By',
 u'Side',
 u'folder',
 u'Well',
 u"'s",
 u'explained',
 u'pretty',
 u'well',
 u'All',
 u'components',
 u'operating',
 u'system',
 u'found',
 u'WinSxS',
 u'folder',
 u'\\u2013',
 u'fact',
 u'call',
 u'location',
 u'component',
 u'store',
 u'Each',
 u'component',
 u'unique',
 u'name',
 u'includes',
 u'version',
 u'language',
 u'processor',
 u'architecture',
 u'built',
 u'The',
 u'WinSxS',
 u'folder',
 u'location',
 u'component',
 u'found',
 u'system',
 u'instances',
 u'files',
 u'see',
 u'system',
 u'\\u201cprojected\\u201d',
 u'hard',
 u'linking',
 u'component',
 u'store',
 u'Let',
 u'repeat',
 u'last',
 u'point',
 u'\\u2013',
 u'one',
 u'i

time: 289 ms


In [10]:
for vector in posts_corpus: # load one vector into memory at a time
    print(vector)


10
[(0, 2), (1, 1), (2, 5), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 9), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 3), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 3), (42, 1), (43, 1), (44, 1), (45, 1), (46, 6), (47, 7), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 2), (65, 1), (66, 1), (67, 1), (68, 1), (69, 9), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 1), (78, 6), (79, 2), (80, 1), (81, 3), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 2), (95, 2), (96, 1), (97, 9), (98, 2), (99, 1), (100, 1), (101, 3), (102, 1), (103, 3), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 2), (110, 

In [12]:
print ("Number of docs in collection: %s"%(posts_corpus.clip_docs))

Number of docs in collection: 10
time: 2 ms


In [ ]:
?gensim.models.LdaModel

time: 64 ms


In [ ]:
#gensim.models.LdaMulticore
"""
lda = gensim.models.LdaModel(corpus, num_topics=5, 
                            id2word=dictionary, 
                            update_every=5, 
                            chunksize=10000,
                            passes=100)
"""

num_topics = 50

lda = gensim.models.LdaModel(posts_corpus, num_topics=5, 
                            id2word=posts_corpus.dictionary
                                ,passes=2
                                )

10
2
10

In [ ]:
lda.show_topics(50)

In [ ]:
topics_matrix = lda.show_topics(formatted=False, num_words=20)
topics_matrix = np.array(topics_matrix)

topic_words = topics_matrix[:,:,1]
for i in topic_words:
    print([str(word) for word in i])
    print()

In [ ]:
lda.show_topics()